## Entrega Final Diplomando Data Science
- Erick Alexei Cambray Servin


In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import shapiro, bartlett
import statsmodels.api as sm

import os 
sns.set()

pd.options.display.max_columns = None

Teniendo esta información vamos a poder hacer una nueva columna la cual vamos a hacer una columna de clasificación sobre la que se hará el análisis de severidad, crearemos una columna que diga 1 si es que la severidad está entre 1 y 2. En 0 si es que está en 3 y 4. Lo que nos dirá que tan grave es un accidente dependiendo de las condiciones en las que sucedió el accidente

In [2]:
dataPath = '/Users/admin/Documents/Personal/data/'

In [3]:
data = pd.read_csv(dataPath + '/US_Accidents_Dec21_updated.csv')

In [4]:
data.head()

,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,Number,Street,Side,City,County,State,Zipcode,Country,Timezone,Airport_Code,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,3,2016-02-08 00:37:08,2016-02-08 06:37:08,40.108910,-83.092860,40.112060,-83.031870,3.230,Between Sawmill Rd/Exit 20 and OH-315/Olentang...,NaN,Outerbelt E,R,Dublin,Franklin,OH,43017,US,US/Eastern,KOSU,2016-02-08 00:53:00,42.1,36.1,58.0,29.76,10.0,SW,10.4,0.00,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,2,2016-02-08 05:56:20,2016-02-08 11:56:20,39.865420,-84.062800,39.865010,-84.048730,0.747,At OH-4/OH-235/Exit 41 - Accident.,NaN,I-70 E,R,Dayton,Montgomery,OH,45424,US,US/Eastern,KFFO,2016-02-08 05:58:00,36.9,NaN,91.0,29.68,10.0,Calm,NaN,0.02,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Night
2,A-3,2,2016-02-08 06:15:39,2016-02-08 12:15:39,39.102660,-84.524680,39.102090,-84.523960,0.055,At I-71/US-50/Exit 1 - Accident.,NaN,I-75 S,R,Cincinnati,Hamilton,OH,45203,US,US/Eastern,KLUK,2016-02-08 05:53:00,36.0,NaN,97.0,29.70,10.0,Calm,NaN,0.02,Overcast,False,False,False,False,True,False,False,False,False,False,False,False,False,Night,Night,Night,Day
3,A-4,2,2016-02-08 06:51:45,2016-02-08 12:51:45,41.062130,-81.537840,41.062170,-81.535470,0.123,At Dart Ave/Exit 21 - Accident.,NaN,I-77 N,R,Akron,Summit,OH,44311,US,US/Eastern,KAKR,2016-02-08 06:54:00,39.0,NaN,55.0,29.65,10.0,Calm,NaN,NaN,Overcast,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Day,Day
4,A-5,3,2016-02-08 07:53:43,2016-02-08 13:53:43,39.172393,-84.492792,39.170476,-84.501798,0.500,At Mitchell Ave/Exit 6 - Accident.,NaN,I-75 S,R,Cincinnati,Hamilton,OH,45217,US,US/Eastern,KLUK,2016-02-08 07:53:00,37.0,29.8,93.0,29.69,10.0,WSW,10.4,0.01,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Day,Day,Day,Day


In [5]:
data['Grado_Severidad'] = [1 if (x==1 or x==2) else 0 for x in data['Severity']]

In [6]:
data.shape

(2845342, 48)

In [7]:
data.isna().sum()

ID                             0
Severity                       0
Start_Time                     0
End_Time                       0
Start_Lat                      0
Start_Lng                      0
End_Lat                        0
End_Lng                        0
Distance(mi)                   0
Description                    0
Number                   1743911
Street                         2
Side                           0
City                         137
County                         0
State                          0
Zipcode                     1319
Country                        0
Timezone                    3659
Airport_Code                9549
Weather_Timestamp          50736
Temperature(F)             69274
Wind_Chill(F)             469643
Humidity(%)                73092
Pressure(in)               59200
Visibility(mi)             70546
Wind_Direction             73775
Wind_Speed(mph)           157944
Precipitation(in)         549458
Weather_Condition          70636
Amenity   

In [8]:
data.drop(['Number', 'Country', 'Amenity'], axis= 1, inplace= True)
data.dropna(subset= ['Zipcode', 'Sunrise_Sunset',
                    'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight'], axis=0, inplace=True)

In [9]:
data.columns

Index(['ID', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng',
       'End_Lat', 'End_Lng', 'Distance(mi)', 'Description', 'Street', 'Side',
       'City', 'County', 'State', 'Zipcode', 'Timezone', 'Airport_Code',
       'Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)',
       'Pressure(in)', 'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)',
       'Precipitation(in)', 'Weather_Condition', 'Bump', 'Crossing',
       'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station',
       'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop',
       'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight', 'Grado_Severidad'],
      dtype='object')

In [10]:
um = ['ID']
trgt = ['Grado_Severidad']

cont_var = ['Distance(mi)', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)',
       'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)', 'Precipitation(in)']

cat_var = ['City','State' ,'Zipcode', 'Timezone', 'Airport_Code',
            'Wind_Direction', 'Weather_Condition','Sunrise_Sunset',
            'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight']


bin_var = ['Bump', 'Crossing', 'Give_Way', 'Junction',
            'No_Exit', 'Railway', 'Roundabout', 'Station',
            'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop']

## Imputaciones

In [11]:
from sklearn.impute import SimpleImputer

for col in cont_var:
    data[col] = SimpleImputer(strategy= 'mean').fit_transform(data[[col]])

In [12]:
for col in cat_var:
    print(col)
    data[col] = SimpleImputer(strategy= 'most_frequent').fit_transform(data[[col]])

City
State
Zipcode
Timezone
Airport_Code
Wind_Direction
Weather_Condition
Sunrise_Sunset
Civil_Twilight
Nautical_Twilight
Astronomical_Twilight


In [13]:
data.dropna(subset= bin_var, axis=0, inplace= True)

In [14]:
data.sample(frac= 0.5).to_csv(dataPath + 'data_clean_proyecto.csv', index= False)

In [15]:
data = pd.read_csv(dataPath + 'data_clean_proyecto.csv')

### Normalización discreta

In [16]:
def normalizar(df,v,umbral):
    aux = df[v].value_counts(True).to_frame()
    aux[f'n_{v}'] = np.where( aux[v] < umbral , 'Otro' , aux.index )

    moda = aux.head(1)[f'n_{v}'].values[0]

    if aux.loc[aux[f'n_{v}'] == 'Otro' ][v].sum() < umbral:
        aux[f'n_{v}'].replace( {'Otro':moda} , inplace=True)

    aux.drop(v, axis=1, inplace=True)
    aux.reset_index(inplace=True)

    return df.merge(aux,left_on=[v], right_on='index', how='inner').drop('index',axis=1)

In [17]:
for v in cat_var:
    data = normalizar(data,v,0.05)

for v in bin_var:
    data = normalizar(data,v,0.05)


In [18]:
varn = [x for x in data.columns if x[0:2] == 'n_']
varn

['n_City',
 'n_State',
 'n_Zipcode',
 'n_Timezone',
 'n_Airport_Code',
 'n_Wind_Direction',
 'n_Weather_Condition',
 'n_Sunrise_Sunset',
 'n_Civil_Twilight',
 'n_Nautical_Twilight',
 'n_Astronomical_Twilight',
 'n_Bump',
 'n_Crossing',
 'n_Give_Way',
 'n_Junction',
 'n_No_Exit',
 'n_Railway',
 'n_Roundabout',
 'n_Station',
 'n_Stop',
 'n_Traffic_Calming',
 'n_Traffic_Signal',
 'n_Turning_Loop']

In [19]:
unarias = [v for v,conteo in zip( varn , [data[v].unique().shape[0] for v in varn] ) if conteo == 1]
varn = [v for v in varn  if v not in unarias]
unarias

['n_City',
 'n_Zipcode',
 'n_Airport_Code',
 'n_Bump',
 'n_Give_Way',
 'n_No_Exit',
 'n_Railway',
 'n_Roundabout',
 'n_Station',
 'n_Stop',
 'n_Traffic_Calming',
 'n_Turning_Loop']

## Normalización Continua

In [20]:
from sklearn.preprocessing import KBinsDiscretizer

def discretizar(df, v, k):

    kb = KBinsDiscretizer( n_bins = k , encode='ordinal', strategy='quantile')
    kb.fit(df[[v]])
    df[f'd_{v}_{k}'] = pd.cut( df[v] , bins=kb.bin_edges_[0] , include_lowest=True ).astype(str)

    return df

In [21]:
for v in cont_var:
    for k in range(3,5):
        data = discretizar(data, v, k)

/Users/admin/Documents/Personal/personal_env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:216: FutureWarning: In version 1.3 onwards, subsample=2e5 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(
/Users/admin/Documents/Personal/personal_env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:216: FutureWarning: In version 1.3 onwards, subsample=2e5 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(
/Users/admin/Documents/Personal/personal_env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:216: FutureWarning: In version 1.3 onwards, subsample=2e5 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warning

In [22]:
vari =[x for x in data.columns if x[0:2] == 'd_']
vari

['d_Distance(mi)_3',
 'd_Distance(mi)_4',
 'd_Temperature(F)_3',
 'd_Temperature(F)_4',
 'd_Wind_Chill(F)_3',
 'd_Wind_Chill(F)_4',
 'd_Humidity(%)_3',
 'd_Humidity(%)_4',
 'd_Pressure(in)_3',
 'd_Pressure(in)_4',
 'd_Visibility(mi)_3',
 'd_Visibility(mi)_4',
 'd_Wind_Speed(mph)_3',
 'd_Wind_Speed(mph)_4',
 'd_Precipitation(in)_3',
 'd_Precipitation(in)_4']

## Information Value

In [23]:
def calculo_iv( df, v, tgt ):
    aux = df.pivot_table( index = v, 
                         columns= tgt, 
                         values = um[0], 
                         aggfunc= 'count', 
                         fill_value=0 )

    aux[list( range(2) ) ] = aux/aux.apply(np.sum)

    aux['w'] = np.log( aux[0] / aux[1] )
    aux['iv'] = (aux[0] - aux[1]) * aux['w']

    return v, aux['iv'].sum()

### Primera partcion IV

In [24]:
from sklearn.model_selection import train_test_split

Xt, Xv, yt, yv = train_test_split(data[vari + um], data[trgt + um])
Xt = Xt.merge( yt , on=um, how='inner').reset_index(drop=True)

In [25]:
# iv de discretizadas -> vari
iv = pd.DataFrame( map( lambda v: calculo_iv( data, v, trgt  ) , vari ) , 
                  columns=['variable','iv'] )

iv['raiz'] = iv['variable'].map( lambda x: '_'.join(x.split('_')[1:2] ) )
iv = iv.sort_values(by=['raiz','iv'],ascending=[1,0]).reset_index(drop=True)

# Eliminación de ivs infs
iv = iv.loc[iv['iv'] != np.inf].reset_index(drop=True)
iv['id'] = iv.groupby('raiz').cumcount()+1
iv = iv.loc[iv.id == 1]

iv.drop(columns=['raiz','id'],inplace=True)

In [26]:
besti = iv['variable'].tolist()

len(besti) , besti

(7,
 ['d_Distance(mi)_4',
  'd_Humidity(%)_4',
  'd_Precipitation(in)_4',
  'd_Pressure(in)_4',
  'd_Temperature(F)_4',
  'd_Visibility(mi)_3',
  'd_Wind_Chill(F)_4'])

## Segunda partición

In [27]:
Xt, Xv, yt, yv = train_test_split(data[besti+ varn + um], data[trgt + um])
Xt = Xt.merge( yt , on=um, how='inner').reset_index(drop=True)

In [28]:
ivd = pd.DataFrame( map( lambda v: calculo_iv(data, v, trgt) , varn ) , 
                   columns=['variable','iv'] )

iv = pd.concat([ iv, ivd ], ignore_index=True)
iv.sort_values(by='iv',ascending=False)

,variable,iv
10,n_Weather_Condition,0.461806
7,n_State,0.371766
6,d_Wind_Chill(F)_4,0.310806
8,n_Timezone,0.243965
9,n_Wind_Direction,0.142248
3,d_Pressure(in)_4,0.084558
0,d_Distance(mi)_4,0.083147
16,n_Junction,0.040107
5,d_Visibility(mi)_3,0.010331
4,d_Temperature(F)_4,0.009622


In [29]:
best = iv['variable'].tolist()
best

['d_Distance(mi)_4',
 'd_Humidity(%)_4',
 'd_Precipitation(in)_4',
 'd_Pressure(in)_4',
 'd_Temperature(F)_4',
 'd_Visibility(mi)_3',
 'd_Wind_Chill(F)_4',
 'n_State',
 'n_Timezone',
 'n_Wind_Direction',
 'n_Weather_Condition',
 'n_Sunrise_Sunset',
 'n_Civil_Twilight',
 'n_Nautical_Twilight',
 'n_Astronomical_Twilight',
 'n_Crossing',
 'n_Junction',
 'n_Traffic_Signal']

## WOE

In [30]:
Xt, Xv, yt, yv = train_test_split(data[best + um], data[trgt + um])

print(Xt.shape, Xv.shape, yt.shape[0], yv.shape[0])

Xt = Xt.merge( yt , on=um, how='inner').reset_index(drop=True)
Xv = Xv.merge( yv , on=um, how='inner').reset_index(drop=True)

(1065444, 19) (355148, 19) 1065444 355148


In [31]:
def codificacion_woe(df, v, tgt):
    aux = df.pivot_table( index=v,
                          columns=tgt,
                          values=um[0],
                          aggfunc='count',
                          fill_value=0)

    aux[list( range(2) )] = aux / aux.apply(np.sum)

    aux['w'] = np.log( aux[0] / aux[1] )

    aux.drop(range(2) , axis=1 , inplace=True )

    aux = aux.to_dict()['w']
    
    return v, aux

In [32]:
mapa_woe = list( map(  lambda v: codificacion_woe( Xt , v , trgt  ) , best  ) )

In [33]:
for v, mapa in mapa_woe:
    Xt[f'w_{v}'] = Xt[v].replace(mapa)
    Xv[f'w_{v}'] = Xv[v].replace(mapa)

## TAD

In [34]:
X_y = pd.concat([Xt, Xv], axis=0)
woe_var = [x for x in X_y.columns if x[0:2]== 'w_']
woe_var

['w_d_Distance(mi)_4',
 'w_d_Humidity(%)_4',
 'w_d_Precipitation(in)_4',
 'w_d_Pressure(in)_4',
 'w_d_Temperature(F)_4',
 'w_d_Visibility(mi)_3',
 'w_d_Wind_Chill(F)_4',
 'w_n_State',
 'w_n_Timezone',
 'w_n_Wind_Direction',
 'w_n_Weather_Condition',
 'w_n_Sunrise_Sunset',
 'w_n_Civil_Twilight',
 'w_n_Nautical_Twilight',
 'w_n_Astronomical_Twilight',
 'w_n_Crossing',
 'w_n_Junction',
 'w_n_Traffic_Signal']

In [35]:
tad = X_y[um + woe_var + trgt]
tad.head()

,ID,w_d_Distance(mi)_4,w_d_Humidity(%)_4,w_d_Precipitation(in)_4,w_d_Pressure(in)_4,w_d_Temperature(F)_4,w_d_Visibility(mi)_3,w_d_Wind_Chill(F)_4,w_n_State,w_n_Timezone,w_n_Wind_Direction,w_n_Weather_Condition,w_n_Sunrise_Sunset,w_n_Civil_Twilight,w_n_Nautical_Twilight,w_n_Astronomical_Twilight,w_n_Crossing,w_n_Junction,w_n_Traffic_Signal,Grado_Severidad
0,A-2081499,-0.206243,-0.040599,-0.009066,-0.313550,-0.019356,-0.007999,-0.636845,-1.063849,-0.829948,0.217477,-0.740540,0.008457,0.008186,0.005288,0.005878,0.018841,-0.075910,-0.014483,1
1,A-1633638,0.305186,0.104783,-0.009066,-0.313550,0.156530,-0.007999,-0.027285,-1.063849,-0.829948,0.217477,-0.740540,-0.015017,0.008186,0.005288,0.005878,0.018841,-0.075910,-0.014483,1
2,A-890220,0.184387,0.104783,-0.009066,-0.223931,0.156530,-0.007999,-0.027285,0.387531,-0.829948,0.217477,0.499646,-0.015017,0.008186,0.005288,0.005878,0.018841,0.526237,-0.014483,1
3,A-823248,-0.206243,-0.022048,-0.009066,0.430568,-0.070736,-0.007999,-0.657934,-0.708961,0.158917,0.217477,0.040366,0.008457,0.008186,0.005288,0.005878,0.018841,-0.075910,-0.014483,1
4,A-1724502,-0.416983,-0.044073,-0.009066,0.430568,-0.019356,-0.007999,-0.657934,-0.708961,0.158917,0.217477,-0.740540,0.008457,0.008186,0.005288,0.005878,-0.280223,-0.075910,0.132470,1


## Modelado de datos y preparación para modelado

Justificación:

Una vez que tenemos los datos ya limpios y las variables seleccionadas con el uso de la prepación WoE podemos entoces tener modelos que puedan medir dentro de las condiciones climáticas junto con los datos de la zona poder decir si es que los accidentes de alta severidad son probables y con la nueva tecnología en autos tener alertas especiales de conducción si es que las probabilidades aumentaran.


Para poder ver seleccionar las variables del modelo aparte de usar el IV vamos a probar los supuestos que nos permitirían modelar con el algoritmo de regresión logística.

In [36]:
X = tad[woe_var].copy()
y = tad['Grado_Severidad']
y.value_counts(normalize = True)

1    0.899525
0    0.100475
Name: Grado_Severidad, dtype: float64

Tenemos una clase desbalanceada pero para eso se pueden aplicar técnias de muestreo y en dimensiones

In [37]:
Xt, Xv, yt, yv = train_test_split(X, y, train_size= 0.7, random_state= 0)

Ahora hacemos el submuestreo

In [38]:
from imblearn.under_sampling import TomekLinks

tomlinks = TomekLinks(sampling_strategy='auto', n_jobs= -1)

Xt, yt = tomlinks.fit_resample(Xt, yt)

In [39]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(max_iter= 10_000)

In [40]:
logreg.fit(Xt,yt)
predicted = logreg.predict(Xv)

In [41]:
from sklearn.metrics import classification_report, f1_score, recall_score, precision_recall_curve, accuracy_score

In [42]:
print(classification_report(yv, predicted))

              precision    recall  f1-score   support

           0       0.43      0.04      0.07     42790
           1       0.90      0.99      0.95    383388

    accuracy                           0.90    426178
   macro avg       0.67      0.52      0.51    426178
weighted avg       0.86      0.90      0.86    426178



In [43]:
print(f'Accuracy: {accuracy_score(y_true= yv, y_pred= predicted)}')
print(f'Recall: {recall_score(y_true= yv, y_pred= predicted)}')
print(f'F1_score: {f1_score(y_true= yv, y_pred= predicted)}')

Accuracy: 0.8982960171571504
Recall: 0.9941730048932153
F1_score: 0.9462000963195027


Cálculo de residuos

In [44]:
residuals = yv - predicted
correlation_matrix = np.corrcoef(residuals, rowvar=False)
print("Correlation matrix:\n", correlation_matrix)

Correlation matrix:
 1.0


Verificamos la normalidad de los residuos

In [45]:
shapiro_test = shapiro(residuals)
print("Shapiro-Wilk test: W = %f, p-value = %f" % shapiro_test)

Shapiro-Wilk test: W = 0.372318, p-value = 0.000000


/Users/admin/Documents/Personal/personal_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:1816: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


Con esto vemos que realmente no hay una buena distribución de los errores. Por lo que se tendría que hacer una predicción con un cambio de modelo

In [46]:
from xgboost import XGBClassifier
xgbC = XGBClassifier()

In [47]:
xgbC.fit(Xt, yt)
y_predicted = xgbC.predict(Xv)

print(classification_report(yv, y_predicted))

              precision    recall  f1-score   support

           0       0.61      0.14      0.23     42790
           1       0.91      0.99      0.95    383388

    accuracy                           0.90    426178
   macro avg       0.76      0.57      0.59    426178
weighted avg       0.88      0.90      0.88    426178



In [48]:
print(f'Accuracy: {accuracy_score(y_true= yv, y_pred= y_predicted)}')
print(f'Recall: {recall_score(y_true= yv, y_pred= y_predicted)}')
print(f'F1_score: {f1_score(y_true= yv, y_pred= y_predicted)}')

Accuracy: 0.904692405520698
Recall: 0.989584963535635
F1_score: 0.9491899005263895


Podemos ver que aunque no se cumplían algunos supuestos para poder hacer el uso de la regresión logística aún así los valores que tenemos son relativamente similares con un modelo más potente lo que nos dice que es correcto usar la transformación para el uso de ambos modelos

## Formación de solución de negocio
El principal uso que se le puede dar a la información obtenida se puede hacer en carros con conexión a internet una alterta de manejo para poder tener más cuidado en las partes donde se tenga probabilidades de tener un choque Grave entonces tener medidas adicionales.

In [51]:
xgbC.predict_proba(tad[woe_var])

array([[0.02107304, 0.97892696],
       [0.0109753 , 0.9890247 ],
       [0.15891993, 0.84108007],
       ...,
       [0.01960826, 0.98039174],
       [0.06798071, 0.9320193 ],
       [0.03434682, 0.9656532 ]], dtype=float32)

In [53]:
tad.loc[:,'proba'] = xgbC.predict_proba(tad[woe_var])[:,1]

/var/folders/4y/6562hjjn49d3rykhmq_11kx00000gn/T/ipykernel_17632/1906876905.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tad.loc[:,'proba'] = xgbC.predict_proba(tad[woe_var])[:,1]


Primero hay que regresar los valores de WoE a sus originales para ser más entendible

In [56]:
reverse_woe = []
for key, dic in mapa_woe:
    reversed_dic = {v: k for k, v in dic.items()}
    reverse_woe.append((key, reversed_dic))

In [60]:
mapa_woe

[('d_Distance(mi)_4',
  {'(-0.001, 0.052]': -0.20624290364277992,
   '(0.052, 0.244]': -0.416982668704957,
   '(0.244, 0.764]': 0.3051856790342143,
   '(0.764, 152.543]': 0.18438725260664196}),
 ('d_Humidity(%)_4',
  {'(0.999, 49.0]': -0.04407282022431994,
   '(49.0, 66.0]': -0.022048183223393532,
   '(66.0, 83.0]': -0.040599170218882354,
   '(83.0, 100.0]': 0.10478300522423036}),
 ('d_Precipitation(in)_4',
  {'(-0.001, 0.00702]': -0.009066120158634866,
   '(0.00702, 24.0]': 0.11913345785059712}),
 ('d_Pressure(in)_4',
  {'(0.019, 29.33]': -0.3135499360994853,
   '(29.33, 29.81]': -0.22393106156652576,
   '(29.81, 30.01]': 0.002609584054834476,
   '(30.01, 56.54]': 0.43056824353552847}),
 ('d_Temperature(F)_4',
  {'(-77.801, 50.0]': 0.15653007831806418,
   '(50.0, 63.0]': -0.0913416736401749,
   '(63.0, 76.0]': -0.01935649545977133,
   '(76.0, 170.6]': -0.07073624681638467}),
 ('d_Visibility(mi)_3',
  {'(-0.001, 10.0]': -0.00799920221756014,
   '(10.0, 130.0]': 1.2800066445058125}),
 (

In [61]:
tad

,ID,w_d_Distance(mi)_4,w_d_Humidity(%)_4,w_d_Precipitation(in)_4,w_d_Pressure(in)_4,w_d_Temperature(F)_4,w_d_Visibility(mi)_3,w_d_Wind_Chill(F)_4,w_n_State,w_n_Timezone,w_n_Wind_Direction,w_n_Weather_Condition,w_n_Sunrise_Sunset,w_n_Civil_Twilight,w_n_Nautical_Twilight,w_n_Astronomical_Twilight,w_n_Crossing,w_n_Junction,w_n_Traffic_Signal,Grado_Severidad,proba
0,A-2081499,-0.206243,-0.040599,-0.009066,-0.313550,-0.019356,-0.007999,-0.636845,-1.063849,-0.829948,0.217477,-0.740540,0.008457,0.008186,0.005288,0.005878,0.018841,-0.075910,-0.014483,1,0.978927
1,A-1633638,0.305186,0.104783,-0.009066,-0.313550,0.156530,-0.007999,-0.027285,-1.063849,-0.829948,0.217477,-0.740540,-0.015017,0.008186,0.005288,0.005878,0.018841,-0.075910,-0.014483,1,0.989025
2,A-890220,0.184387,0.104783,-0.009066,-0.223931,0.156530,-0.007999,-0.027285,0.387531,-0.829948,0.217477,0.499646,-0.015017,0.008186,0.005288,0.005878,0.018841,0.526237,-0.014483,1,0.841080
3,A-823248,-0.206243,-0.022048,-0.009066,0.430568,-0.070736,-0.007999,-0.657934,-0.708961,0.158917,0.217477,0.040366,0.008457,0.008186,0.005288,0.005878,0.018841,-0.075910,-0.014483,1,0.946170
4,A-1724502,-0.416983,-0.044073,-0.009066,0.430568,-0.019356,-0.007999,-0.657934,-0.708961,0.158917,0.217477,-0.740540,0.008457,0.008186,0.005288,0.005878,-0.280223,-0.075910,0.132470,1,0.979009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355143,A-607151,-0.416983,-0.040599,-0.009066,-0.313550,0.156530,-0.007999,-0.027285,0.387531,0.158917,0.217477,0.040366,0.008457,0.008186,0.005288,0.005878,0.018841,-0.075910,-0.014483,1,0.945989
355144,A-1866177,0.184387,0.104783,-0.009066,-0.313550,0.156530,-0.007999,-0.027285,0.387531,0.158917,-0.752323,-0.740540,-0.015017,-0.017467,-0.014115,-0.019416,0.018841,-0.075910,-0.014483,1,0.920899
355145,A-592142,0.184387,-0.044073,-0.009066,-0.313550,-0.070736,-0.007999,-0.657934,-1.063849,-0.829948,0.217477,-0.740540,0.008457,0.008186,0.005288,0.005878,0.018841,-0.075910,-0.014483,1,0.980392
355146,A-1983439,-0.416983,0.104783,-0.009066,0.430568,-0.019356,-0.007999,-0.636845,0.387531,0.158917,0.217477,0.499646,-0.015017,-0.017467,-0.014115,-0.019416,0.018841,-0.075910,-0.014483,1,0.932019


In [59]:
reverse_woe

[('d_Distance(mi)_4',
  {-0.20624290364277992: '(-0.001, 0.052]',
   -0.416982668704957: '(0.052, 0.244]',
   0.3051856790342143: '(0.244, 0.764]',
   0.18438725260664196: '(0.764, 152.543]'}),
 ('d_Humidity(%)_4',
  {-0.04407282022431994: '(0.999, 49.0]',
   -0.022048183223393532: '(49.0, 66.0]',
   -0.040599170218882354: '(66.0, 83.0]',
   0.10478300522423036: '(83.0, 100.0]'}),
 ('d_Precipitation(in)_4',
  {-0.009066120158634866: '(-0.001, 0.00702]',
   0.11913345785059712: '(0.00702, 24.0]'}),
 ('d_Pressure(in)_4',
  {-0.3135499360994853: '(0.019, 29.33]',
   -0.22393106156652576: '(29.33, 29.81]',
   0.002609584054834476: '(29.81, 30.01]',
   0.43056824353552847: '(30.01, 56.54]'}),
 ('d_Temperature(F)_4',
  {0.15653007831806418: '(-77.801, 50.0]',
   -0.0913416736401749: '(50.0, 63.0]',
   -0.01935649545977133: '(63.0, 76.0]',
   -0.07073624681638467: '(76.0, 170.6]'}),
 ('d_Visibility(mi)_3',
  {-0.00799920221756014: '(-0.001, 10.0]',
   1.2800066445058125: '(10.0, 130.0]'}),
 (

In [67]:
final = pd.DataFrame()
for v, mapa in reverse_woe:
    final[v] = tad[f'w_{v}'].replace(mapa)
final['target'] = tad['Grado_Severidad']

final.head()

In [ ]:
sns.catplot(data=df, kind="strip", hue="target",
            x="col_1", y="col_2", jitter=True)